# Error Learning

## Data Construction

In [1]:
import numpy as np
import os
import pickle
from pathlib import Path
import wandb

def load_and_process_data(run_id):
    file_path = f"rom_tracking_data/{run_id}/dataset.pickle"

    # Load data from the pickle file
    with open(file_path, "rb") as f:
        epoch_data = pickle.load(f)
        print("Keys in dataset:", epoch_data.keys())

    processed_data = {}
    for key in epoch_data:
        processed_data[key] = epoch_data[key][:-1, :]

    processed_data['e'] = processed_data['pz_x'] - processed_data['z']
    processed_data['e_p1'] = np.roll(processed_data['e'], -1, axis=0)
    processed_data['e_p1'][-1, :] = 0 # for wrap around...

    return processed_data

def save_data(run_id, data):
    output_path = Path('rom_tracking_data', run_id, "processed_data_with_errors.pickle")
    with open(output_path, "wb") as f:
        pickle.dump(data, f)
    print(f"Data processing and saving complete. Data saved to {output_path}")

# Usage example
run_id = "single_integrator_o9zc48x6"
data = load_and_process_data(run_id)
save_data(run_id, data)


Keys in dataset: dict_keys(['z', 'pz_x', 'v', 'z_p1', 'pz_x_p1'])
Data processing and saving complete. Data saved to rom_tracking_data/single_integrator_o9zc48x6/processed_data_with_errors.pickle


## Error Learning

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import argparse
import numpy as np
from pathlib import Path
import wandb

In [3]:
def convert_to_tensor_input(data):
    features = torch.tensor(data['e'], dtype=torch.float32)
    targets = torch.tensor(data['e_p1'], dtype=torch.float32)
    return features, targets

def create_data_loaders(X, y, batch_size=64):
    dataset = TensorDataset(X, y)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    return train_loader, test_loader

In [7]:
# Error Dynamics Model
class ErrorDynamicsModel(nn.Module):
    def __init__(self, input_dim, num_units, num_layers):
        super(ErrorDynamicsModel, self).__init__()
        layers = [nn.Linear(input_dim, num_units), nn.ReLU()]
        for _ in range(num_layers - 1):
            layers.extend([nn.Linear(num_units, num_units), nn.ReLU()])
        layers.append(nn.Linear(num_units, input_dim))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

# Custom Loss Function for Error Dynamics
class CustomErrorDynamicsLoss(nn.Module):
    def __init__(self):
        super(CustomErrorDynamicsLoss, self).__init__()

    def forward(self, y_pred, y_true):
        return torch.norm(y_pred - y_true, p=2, dim=1).mean()

# Training and Testing Routine
def train_and_test(model, criterion, optimizer, train_loader, test_loader, num_epochs=50):
    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        for data, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        model.eval()
        total_test_loss = 0
        with torch.no_grad():
            for data, targets in test_loader:
                outputs = model(data)
                loss = criterion(outputs, targets)
                total_test_loss += loss.item()

        # Logging to wandb
        wandb.log({'Epoch': epoch, 'Train Loss': total_train_loss / len(train_loader), 'Test Loss': total_test_loss / len(test_loader)})

        # Save the best model
        if epoch == 0 or total_test_loss < best_test_loss:
            best_test_loss = total_test_loss
            model_path = f"models/model_epoch_{epoch}.pth"
            os.makedirs(os.path.dirname(model_path), exist_ok=True)
            torch.save(model.state_dict(), model_path)

# Main Function
def main(run_id, num_units=100, num_layers=2, num_epochs=500):
    wandb.init()
    
    data = load_and_process_data(run_id)
    X, y = convert_to_tensor_input(data)
    train_loader, test_loader = create_data_loaders(X, y)

    input_dim = X.size(1)
    model = ErrorDynamicsModel(input_dim, num_units, num_layers)
    criterion = CustomErrorDynamicsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    train_and_test(model, criterion, optimizer, train_loader, test_loader, num_epochs)

    wandb.finish()

In [8]:
wandb.login(key="70954bb73c536b7f5b23ef315c7c19b511e8a406")
main('single_integrator_o9zc48x6')

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Epoch,▁█
Test Loss,▁█
Train Loss,█▁
Epoch,1
Test Loss,0.00743
Train Loss,0.00757


Keys in dataset: dict_keys(['z', 'pz_x', 'v', 'z_p1', 'pz_x_p1'])
in train test
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training

training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
trai

training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
trai

...

training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
trai

training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
trai

training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
training...
trai

Exception ignored in: <function WeakSet.__init__.<locals>._remove at 0x7fb4b1daa4c0>
Traceback (most recent call last):
  File "/Users/colejohnson/opt/anaconda3/lib/python3.9/_weakrefset.py", line 39, in _remove
    def _remove(item, selfref=ref(self)):
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--run_id", type=str, required=True, help="Run ID for the data.")
    parser.add_argument("--num_units", type=int, default=100, help="Number of units in each hidden layer.")
    parser.add_argument("--num_layers", type=int, default=2, help="Number of hidden layers.")
    parser.add_argument("--num_epochs", type=int, default=50, help="Number of epochs to train.")
    args = parser.parse_args()
    wandb.login(key="70954bb73c536b7f5b23ef315c7c19b511e8a406")

    main(args.run_id, args.num_units, args.num_layers, args.num_epochs)